In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/data_for_modeling.csv')

In [6]:
import pandas as pd

df = pd.read_csv('data/data_for_modeling.csv')

# 1) Parse the date column (convert strings → datetime)
df['date'] = pd.to_datetime(df['date'])

# 2) Sort by date and compute cutoff
df = df.sort_values('date')
max_date = df['date'].max()
cutoff_date = max_date - pd.Timedelta(days=22)

# 3) Split
train    = df[df['date'] <= cutoff_date]
holdout  = df[df['date'] >  cutoff_date]

print(f"Max date:      {max_date.date()}")
print(f"Cutoff date:   {cutoff_date.date()}")
print(f"Train rows:    {train.shape[0]}")
print(f"Holdout rows:  {holdout.shape[0]}")


Max date:      2014-10-31
Cutoff date:   2014-10-09
Train rows:    231404
Holdout rows:  4634
